# Reinforcement Learning 2021-22: Second Group Assignment

**Deadline**: Mon 14 March 2022, 23:59

|Nr|**Name**|**Student ID**|**Email**|
|--|--------|--------------|---------|
|1.|Zhen Fang|  12542636|   fangzhen0506@yahoo.com       |
|2.| Weiqi Mao       | 12542644             |   12542644@uva.nl|
|3.|        |              |         |

**Declaration of Originality**

We whose names are given under 1., 2., and 3. above declare that:

1. These solutions are solely our own work.
2. We have not made (part of) these solutions available to any other student.
3. We shall not engage in any other activities that will dishonestly improve my results or dishonestly improve or hurt the results of others.

## Instructions for completing and submitting the assignment
Please pay attention to the following instructions:

1. Please follow carefully the steps outlined in the assignment. If you cannot solve an exercise and this hinders continuing with subsequent exercises, try to find a way to work around it and give a clear explanation for the solution you have chosen.
2. Submit your work in the form of a Jupyter notebook and a html file (see point 6 below) via Canvas, before the deadline. Your notebook should not give errors when executed with `Run All`.
3. Most of your answers will consist of code. Make sure your code is well structured, efficient and provided with comments where needed. These aspects will be taken into account in the grading of your work.
4. Sometimes you are asked to answer some (open) questions. Please type your answer in the given Markdown cells and use your own words in answering those questions.
5. You are allowed to work on the assignment in groups of no more than 3 students and to submit together.
6. Save the complete work with all outputs in an HTML file: "File"->"Download as" -> "HTML (.html)" in Jupyter Notebook. Upload this HTML file to canvas as a supplement. You must submit BOTH notebook and HTML files.


## Taxi Problem

In this assignment, you will solve the taxi problem for the following map:

````
        +---------+
        |R: | : :G|
        | : | : : |
        | : : : : |
        | | : | : |
        |Y| : |B: |
        +---------+
````

There are four designated locations in the grid world indicated by R(ed), G(reen), Y(ellow), and B(lue). When the episode starts, the taxi starts off at a random square and the passenger is at a random location. The taxi drives to the passenger's location, picks up the passenger, drives to the passenger's destination (another one of the four specified locations), and then drops off the passenger. Once the passenger is dropped off, the episode ends. The rewards are:

- -1 per step unless other reward is triggered.
- +20 delivering passenger.
- -10 executing "pickup" and "drop-off" actions illegally.

If a navigation action would cause the taxi to hit a wall (solid borders in the map), the action is a no-op, and there is only the usual reward of −1.

In [1]:
import numpy as np
from collections import defaultdict

GroupNumber=26 # Input your group number Here

Let us import the ``TaxiEnv`` class from env_taxi.py and create an instance of the Taxi environment.

In [2]:
from env_taxi import TaxiEnv
env=TaxiEnv()

Then, we reset the environment:

In [3]:
np.random.seed(100)
env.reset()

(2, 3, 2, 1)

It returns four state variables:

- Row index of the taxi (starting from 0 in Python)
- Colum index of the taxi (starting from 0 in Python)
- Passenger location (0-4): 0=R, 1=G, 2=Y, 3=B, 4=in taxi
- Destination location (0-3): same encoding rule as that for passenger location but excluding the "in taxi" option.

You can use the ``describe_state`` method of the Taxi instance to display the state variables without location encoding.

In [4]:
env.describe_state()

{'Taxi Location': [2, 3], 'Passenger Location': 'Y', 'Destination Index': 'G'}

You can use the ``render`` method to visualize the state.

In [5]:
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+



There are 6 discrete deterministic actions:
- 'South': move south
- 'West': move north
- 'East': move east
- 'West': move west
- 'Pickup': pickup passenger
- 'Dropoff': drop off passenger

In [6]:
print(env.action_space)

['South', 'North', 'East', 'West', 'Pickup', 'Dropoff']


Let us move one step to west:

In [7]:
env.step('West')

((2, 2, 2, 1), -1, False)

The output is a 3-tuple: the new state (a list of 4 numbers), reward, and whether the episode is ended. 

Let us visualize the new state. Note that the last action is shown at the bottom as well.

In [8]:
env.render()

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+
  (West)


**Exercise 1** Complete the codes below to generate _one_ episode for the taxi driver who:
- Picks up the passenger when the taxi is at the location of the passenger when they are not yet at the destination;
- Drops off the passenger in the taxi when reaching the destination;
- Moves randomly with equal probabilities when finding the passenger or when the passenger is in the taxi (but not yet at destination). 

Then print the sum of rewards for the taxi driver in your episode.

In [9]:
state=env.reset(271) # The number 271 fixes the initial state. Do not change it.
env.render()
print(env.describe_state())

+---------+
|R: | : :G|
| : | : : |
| : : : : |
| | : | : |
|Y| : |B: |
+---------+

{'Taxi Location': [2, 3], 'Passenger Location': 'Y', 'Destination Index': 'B'}


Let us program the policy function for the taxi driver:

In [10]:
def naive_policy(state):
    ## Start Coding Here
    
    if state[2]!=4:
        if state[0:2]==env.locs[state[2]] and state[0:2]!=env.locs[state[3]]:
            action=env.action_space[4] #Pickup
        else:
            action=np.random.choice(env.action_space[0:4])#randomly move
    if state[2]==4:
        if state[0:2]==env.locs[state[3]]:
            action=env.action_space[5]# Dropoff
        else:
            action=np.random.choice(env.action_space[0:4])#randomly move
            
    ## End Coding Here
    return action

In [11]:
# Import the simulate_episode function from env_taxi.py to help you simulate episode(s)
from env_taxi import simulate_episode
help(simulate_episode)

Help on function simulate_episode in module env_taxi:

simulate_episode(env, policy, max_iter=inf)
    Simulate a episode following a given policy
    @param env: game environment
    @param policy: policy (a dictionary or a function)
    @return: resulting states, actions and rewards for the entire episode



In [12]:
# Set the seed to make results reproducible
np.random.seed(GroupNumber)

states, actions, rewards = simulate_episode(env,naive_policy)

# Save the sum of rewards in this variable G
G=np.sum(rewards)


# Print the return for this episode
print("Return for this episode:", G)

Return for this episode: -426


**Exercise 2** Following the steps below to investigate whether we can solve the optimal policy by using Monte Carlo Exploring Starts. If possible, complete the codes and save the optimal (greedy) policy as a dictionary/defaultdict object. Otherwise, comment on the issues you encounter. 

Follow the instructions below to complete the exercise.

In [13]:
# We import the simulate_episode_ES function from env_taxi.py to simulate episodes with exploring starts
from env_taxi import simulate_episode_ES
help(simulate_episode_ES)


# Let us try to simulate one episode with exploring start for the naive policy from Exercise 1
# Print the states, actions and rewards for an episode (the first 200 steps only)
np.random.seed(GroupNumber)
states, actions, rewards=simulate_episode_ES(env, naive_policy, 200)
print(states, actions, rewards)

Help on function simulate_episode_ES in module env_taxi:

simulate_episode_ES(env, policy, max_iter=inf)
    Simulate a episode with exploring starts
    @param env: game environment
    @param policy: policy (a dictionary or a function)
    @param max_iter: maximum number of time steps
    @return: resulting states, actions and rewards for the entire episode

[(1, 2, 2, 3), (2, 2, 2, 3), (2, 1, 2, 3), (3, 1, 2, 3), (2, 1, 2, 3), (2, 0, 2, 3), (3, 0, 2, 3), (2, 0, 2, 3), (2, 1, 2, 3), (2, 2, 2, 3), (3, 2, 2, 3), (2, 2, 2, 3), (3, 2, 2, 3), (4, 2, 2, 3), (4, 1, 2, 3), (4, 1, 2, 3), (3, 1, 2, 3), (3, 1, 2, 3), (3, 2, 2, 3), (3, 2, 2, 3), (3, 1, 2, 3), (3, 1, 2, 3), (4, 1, 2, 3), (4, 1, 2, 3), (4, 1, 2, 3), (4, 1, 2, 3), (3, 1, 2, 3), (4, 1, 2, 3), (3, 1, 2, 3), (3, 2, 2, 3), (2, 2, 2, 3), (1, 2, 2, 3), (1, 3, 2, 3), (1, 2, 2, 3), (1, 3, 2, 3), (1, 2, 2, 3), (2, 2, 2, 3), (3, 2, 2, 3), (4, 2, 2, 3), (4, 1, 2, 3), (4, 1, 2, 3), (4, 1, 2, 3), (4, 1, 2, 3), (4, 2, 2, 3), (4, 1, 2, 3), (3, 1,

If you can solve the optimal policy by using Monte Carlo Exploring Starts, write down your codes in the next cell and save the optimal (greedy) policy in the dictionary object ``pi_ES`` initialized below. Use no more than 50000 episodes. We will import the ``performance_evaluation`` function from env_taxi.py to evaluate your policy in new episodes (that terminate within 1000 time steps).

In [14]:
# Save your greedy actions as the values of this dictionary, whose keys are the states
pi_ES= defaultdict(lambda: np.random.choice(env.action_space))
np.random.seed(GroupNumber)

## Start Coding Here 

from tqdm import tqdm
G_sum = defaultdict(float)
N = defaultdict(int)
Q = defaultdict(lambda: dict(zip(env.action_space, np.empty(6))))
for episode in tqdm(range(50000)):
    

    states_t, actions_t, rewards_t = simulate_episode_ES(env,naive_policy)
        
    return_t = 0
    G = {}
    for state_t, action_t, reward_t in zip(states_t[::-1][1:], actions_t[::-1], rewards_t[::-1]):
        return_t =return_t + reward_t
        G[(state_t, action_t)] = return_t
        
    for state_action, return_t in G.items():
            state, action = state_action
            G_sum[state_action] += return_t
            N[state_action] += 1
            Q[state][action] = G_sum[state_action] / N[state_action]
            pi_ES[state]=max(Q[state],key=Q[state].get)  #For each state, keep the action with the average largest value. 


## End Coding Here



# We import the performance_evaluation function from env_taxi.py to evaluate your optimal policy
from env_taxi import performance_evaluation
performance_evaluation(env,pi_ES)

100%|██████████| 10000/10000 [04:26<00:00, 37.48it/s]

The sum of rewards per episode: -9993.350868100544
The percentage of episodes that cannot terminate within 1000 steps: 0.6141


If you cannot solve the optimal policy by using Monte Carlo Exploring Starts, explain the issue(s) and why they cannot be addressed.

Answer: Using this policy，most likely an episode cannot terminate within 1000 steps， because the first action is a random choice，the reward of the first action could also influence the average Q,and also because "-1 per step unless other reward is triggered",then it will take much more time for action value to converge, and to determine which action is the best choice for each state.

**Exercise 3**:  Find the optimal policy by using Q-learning with 50000 episodes and the exploration probability $\varepsilon=0.1$. Try two different values of the step-size parameter $\alpha=0.4$ and $\alpha=0.1$. Compare their performance in 10000 new episodes and comment on the similarities or/and differences. 

Follow the instructions below to complete the exercise.

In [16]:
np.random.seed(GroupNumber)

## Start Coding Here: Find the optimal policy by using Q-learning with 50000 episodes

def gen_epsilon_greedy_policy(action_space, Q, epsilon):
    def epsilon_greedy_policy(state):
        greedy_action=max(Q[state],key=Q[state].get)  #choose the action with the largest value.
        random_action=np.random.choice(action_space)  #randomly choose among all the actions.
        return np.random.choice([random_action,greedy_action],p=[epsilon,1-epsilon]) 
    return epsilon_greedy_policy
def Q_learning(env, gamma, n_episode, alpha, epsilon):
    length_episode = np.zeros(n_episode) 
    sum_reward_episode = np.zeros(n_episode)
    Q = defaultdict(lambda: dict(zip(env.action_space, np.zeros(len(env.action_space)))))
    epsilon_greedy_policy = gen_epsilon_greedy_policy(env.action_space, Q, epsilon)
    pi= defaultdict(lambda: np.random.choice(env.action_space))
    for episode in tqdm(range(n_episode)):
        state = env.reset()
        is_done = False
        action = epsilon_greedy_policy(state)
        while not is_done:
            next_state, reward, is_done = env.step(action)
            next_action = epsilon_greedy_policy(state)
           
            v= max(Q[next_state].values())
            
            td_delta = reward + gamma * v - Q[state][action]
            
            Q[state][action] += alpha * td_delta
            # Updte Target Policy
            pi[state]=max(Q[state],key=Q[state].get)
            
            length_episode[episode] += 1
            sum_reward_episode[episode] += reward
            if is_done:
                break     
            state = next_state
            action = next_action
    return pi
# Save the policy for alpha=0.4 as a Python dictionary/defaultdic object pi_qlearning_1
# Save the policy for alpha=0.1 as a Python dictionary/defaultdic object pi_qlearning_2

n_episode=50000
gamma=1
pi_qlearning_1=Q_learning(env, gamma, n_episode, alpha=0.4, epsilon=0.1)
pi_qlearning_2=Q_learning(env, gamma, n_episode, alpha=0.1, epsilon=0.1)

## End Coding Here

100%|██████████| 50000/50000 [01:57<00:00, 425.03it/s]


In [26]:
# Compare their performance in 10000 new episodes: Do you find a big difference?
from env_taxi import performance_evaluation

print('--------  Q-learning, alpha=0.4  --------')
performance_evaluation(env,pi_qlearning_1)

print('--------  Q-learning, alpha=0.1  --------')
performance_evaluation(env,pi_qlearning_2)

  2%|▏         | 208/10000 [00:00<00:04, 2073.39it/s]

--------  Q-learning, alpha=0.4  --------


  5%|▍         | 453/10000 [00:00<00:04, 2258.34it/s]

The sum of rewards per episode: 7.8886
The percentage of episodes that cannot terminate within 1000 steps: 0.0
--------  Q-learning, alpha=0.1  --------


100%|██████████| 10000/10000 [00:04<00:00, 2228.01it/s]

The sum of rewards per episode: 7.8886
The percentage of episodes that cannot terminate within 1000 steps: 0.0


Commment on the similarities or/and differences.

Answer:Using either of these two policies could an  episode terminate within 1000 steps, the sum of rewards per episode are the same.

**Exercise 4**: Follow the steps below to find the optimal $\varepsilon$-soft policy by using Sarsa with 50000 episodes and the exploration probability $\varepsilon=0.1$. Try two different values of the step-size parameter $\alpha=0.4$ and $\alpha=0.1$. Compare their performance, and comment on the similarties or/and differences with that of Q-learning.

Follow the instructions below to complete the exercise.

In [17]:
np.random.seed(GroupNumber)

## Start Coding Here
def gen_epsilon_greedy_policy(action_space, Q, epsilon):
    def epsilon_greedy_policy(state):
        greedy_action=max(Q[state],key=Q[state].get)
        random_action=np.random.choice(action_space)
        return np.random.choice([random_action,greedy_action],p=[epsilon,1-epsilon]) 
    return epsilon_greedy_policy
def sarsa(env, gamma, n_episode, alpha, epsilon):
    
    length_episode = np.zeros(n_episode) 
    sum_reward_episode = np.zeros(n_episode)
    
    Q = defaultdict(lambda: dict(zip(env.action_space, np.zeros(len(env.action_space)))))
    
    pi = gen_epsilon_greedy_policy(env.action_space, Q, epsilon)
   
    
    for episode in tqdm(range(n_episode)):
        
        ## Start Coding Here
        state = env.reset()
        is_done = False
        action = pi(state)
        while not is_done:
            next_state, reward, is_done = env.step(action)
            next_action = pi(next_state)
            td_delta = reward + gamma * Q[next_state][next_action] - Q[state][action]
            Q[state][action] += alpha * td_delta
            
            length_episode[episode] += 1
            sum_reward_episode[episode] += reward
            if is_done:
                break
            state = next_state
            action = next_action
            
        ## End Coding Here
            
    return Q,pi
# Save the policy for alpha=0.4 in a Python function pi_sarsa_1
# Save the policy for alpha=0.1 in a Python function pi_sarsa_2
n_episode=50000
epsilon=0.1
alpha=0.4
gamma=1
Q,pi_sarsa_1=sarsa(env, gamma, n_episode, alpha, epsilon)
alpha=0.1
Q_b,pi_sarsa_2=sarsa(env, gamma, n_episode, alpha, epsilon)
# End Coding Here



100%|██████████| 50000/50000 [01:01<00:00, 816.10it/s]


In [25]:
from env_taxi import performance_evaluation

print('--------  Sarsa, alpha=0.4  --------')
performance_evaluation(env,pi_sarsa_1)

print('--------  Sarsa, alpha=0.1  --------')
performance_evaluation(env,pi_sarsa_2)

  1%|          | 63/10000 [00:00<00:38, 259.50it/s]

--------  Sarsa, alpha=0.4  --------


  2%|▏         | 206/10000 [00:00<00:09, 1037.58it/s]

The sum of rewards per episode: -36.84618461846185
The percentage of episodes that cannot terminate within 1000 steps: 0.0001
--------  Sarsa, alpha=0.1  --------


100%|██████████| 10000/10000 [00:09<00:00, 1040.68it/s]

The sum of rewards per episode: 2.3004
The percentage of episodes that cannot terminate within 1000 steps: 0.0


Compare their performance, and comment on the similarties or/and differences with that of Q-learning.

Answer:When using these two Sarsa methods, an episode could be very likely to terminate within 1000 steps,but alpha=0.4，there is an very small probability that an episode can not terminate with 1000 steps .Also the different alpha leads to quite different sum of rewards.Compared with Q-learning methods, these two sum of rewards are not statisfying.And the choice of alpha seems have a great effect on the results of Sarsa,while for Q-learning, alpha could change nothing.

**Exercise 5**: Do you expect double Q-learning will improve Q-learning substantially? Motivate your answer. You do not need to implement the double Q-learning algorithm.

Answer:Yes,we do.Using double Q-learning,we could expect that the effect of extreme action value will decrease,and the estimation error will decrease，which means eliminating the maximization bias.

**Exercise 6** 
Follow the steps below to investigate whether we can solve the optimal policy by using off-policy Monte Carlo control with weighted importance sampling. Use the optimal $\varepsilon$-soft policy found by Sarsa above with $\alpha=0.1$ as the behavior policy and use no more than 50000 episodes.

In [27]:
np.random.seed(GroupNumber)

## Start Coding Here
def mc_control_off_policy_weighted(env, gamma, n_episode):
    
    n_action = len(env.action_space)
    C = defaultdict(float)
    
    # Create a function to generate action based on the behavior policy
    b_func=pi_sarsa_2
    
    # Initialization
    Q = defaultdict(lambda: dict(zip(env.action_space, np.empty(6))))
    pi= defaultdict(lambda: np.random.choice(env.action_space))
    
    for episode in tqdm(range(n_episode)):
        states_t, actions_t, rewards_t = simulate_episode(env, b_func)
        
        ## Start Coding Here
        W = 1.
        return_t = 0.
        for state_t, action_t, reward_t in zip(states_t[::-1][1:], actions_t[::-1], rewards_t[::-1]):
            return_t = gamma * return_t + reward_t
            C[(state_t, action_t)] += W
            Q[state_t][action_t] += (W / C[(state_t, action_t)]) * (return_t - Q[state_t][action_t])
            pi[state_t]=max(Q[state_t],key=Q[state_t].get)
            
            if action_t != pi[state_t]:
                break
            if action_t in max(Q[state_t],key=Q[state].get):
                b=((1-epsilon)/len(max(Q[state_t],key=Q[state].get))+epsilon/len(env.action_space))
            else:
                b=epsilon/len(env.action_space)
            W *= 1./ b
        ## End Coding Here
                
    return  pi
# Save your optimal policy as a dictionary/defaultdict Python object pi_wis
n_episode=50000


pi_wis=mc_control_off_policy_weighted(env, gamma, n_episode)


## End Coding Here

100%|██████████| 50000/50000 [00:51<00:00, 971.94it/s] 


Check the total number of distinct states visited in your episodes. Have you reached all 400 possible states (within the episode)?

In [21]:
print('Number of States Visited:', len(pi_wis)) #less than 400

Number of States Visited: 354


Generate a complete policy that:
- follows your optimal policy (in this exercise) if the state is visited in your episodes (in this exercise);
- otherwise follows the naive policy in Exercise 1.

If you have reached all possible states in your episodes, it is identical to your optimal policy.

In [23]:
def complete_policy(state):
    ## Start Coding Here
    if state in pi_wis:
        action=pi_wis[state]
    else:
        action=naive_policy(state) 
    ## End Coding Here
    return action

Let us evaluate its performance:

In [24]:
performance_evaluation(env,complete_policy)

100%|██████████| 10000/10000 [00:04<00:00, 2067.45it/s]

The sum of rewards per episode: 6.8081
The percentage of episodes that cannot terminate within 1000 steps: 0.0


**Exercise 7**: What are your conclusions from the analysis above? What are the advantages and disadvantages of these methods?

Answer:From all the policy evaluations, we can conclude that Q-learning has the highest reward among all methods. While Monte Carlo Exploring Starts gives the worst reward which more than half of episodes can not end within 1000 steps. Q-learing is the most informative method for this particular question. However, Q-learning takes longer time since the length per episode is longer than length of SARSA. And SARSA gives a lower rewards but gives a faster route.Using off-policy Monte Carlo control with weighted importance sampling and combined with the naive policy could also get an satisfactory result,and it takes a short time.